In [29]:
import pandas as pd
import lightgbm as lgb # The new weapon
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.multioutput import MultiOutputRegressor

train_df = pd.read_csv('/kaggle/input/training/train.csv',skiprows=0)

Y = train_df.filter(like='Blend')

X = train_df.drop(Y.columns , axis = 1)


In [30]:
def feature_expansion(X):

    x_engg = X.copy()
    new_features = []
    
    for i in range(1,6):
        fractionc = f"Component{i}_fraction"
        proportionc = X[fractionc]
        propertyc  = f"Component{i}_Property"
        ComponentPropertiesc = X.filter(like=propertyc)  
        partial_componets = proportionc.values.reshape(-1, 1) * ComponentPropertiesc
    
        partial_componets.columns = [f'weighted_C{i}_P{j}' for j in range(1, 11)]
        
        new_features.append(partial_componets)
    
    X_final = pd.concat([x_engg] + new_features, axis=1 )
    return X_final

In [31]:
SHIFT_VALUE = 10
Y_shifted = Y + SHIFT_VALUE
X_final = feature_expansion(X)
X_train, X_val, y_train_shifted, y_val_shifted = train_test_split(
    X_final, Y_shifted, test_size=0.2, random_state=42
)


core_lgbm = lgb.LGBMRegressor(
    objective='mae',
    n_estimators=1000,
    learning_rate=0.05,
    num_leaves=31,
    n_jobs=-1,
    seed=42
)

In [32]:
multi_output_lgbm = MultiOutputRegressor(core_lgbm)

multi_output_lgbm.fit(X_train, y_train_shifted)

val_predictions_shifted = multi_output_lgbm.predict(X_val) 

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002082 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25727
[LightGBM] [Info] Number of data points in the train set: 1600, number of used features: 105
[LightGBM] [Info] Start training from score 9.985353
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002008 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25727
[LightGBM] [Info] Number of data points in the train set: 1600, number of used features: 105
[LightGBM] [Info] Start training from score 9.985323
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001969 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25727
[LightGBM] [Info] Number of data points in the train set: 1600, number of used features: 105
[LightGBM] [Info] Start 

In [33]:
val_predictions_original = val_predictions_shifted - SHIFT_VALUE
y_val_original = y_val_shifted - SHIFT_VALUE

mape_cost = mean_absolute_percentage_error(y_val_original, val_predictions_original)
print(f"Final Validation MAPE: {mape_cost:.4f}")

reference_cost = 2.72
leaderboard_score = 100 - (90 * mape_cost) / reference_cost
final_score = max(10, leaderboard_score)

print(f"\nScore with LightGBM: {final_score:.2f}")

Final Validation MAPE: 2.3061

Score with LightGBM: 23.70
